In [2]:
# get imports # import files
from threading import Thread
import queue as que
import pandas as pd
import os, pickle
import nltk
import math
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
import nltk
from collections import Counter 

stemmer = SnowballStemmer("english")
cachedStopWords = stopwords.words("english")

In [100]:
#methods
def save_it_all(obj, filename):
    os.makedirs(os.path.dirname(filename), exist_ok=True)
    with open(filename, 'wb') as output:  # Overwrites any existing file.
        pickle.dump(obj, output, pickle.HIGHEST_PROTOCOL)
    
def load_objects(file):
    with open(file, 'rb') as input:
        return pickle.load(input)
    
def merge_two_dicts(x, y):
    z = x.copy()   # start with x's keys and values
    z.update(y)    # modifies z with y's keys and values & returns None
    return z

def seperator(docs):
    print(len(docs))
    txt = []
    typesInfo = []
    docsInfo = []
    for line in docs:
        words = line.split()
        content = (" ").join(words[1:])
        item = (", ").join([words[0], content])
        txt.append(item)
        typesInfo.append(words[0])
        docsInfo.append(content)
    return txt, typesInfo, docsInfo

def write(data):
    print(type(data))
    with open('./training_data.csv', 'w') as f:
        for item in data:
            f.write("%s\n" % item)
            
# methods
def read(fileName):
    with open(fileName) as f:
        content = f.readlines()
    # you may also want to remove whitespace characters like `\n` at the end of each line
    content = [x.strip() for x in content]
    return content

def save_it_all(obj, filename):
    os.makedirs(os.path.dirname(filename), exist_ok=True)
    with open(filename, 'wb') as output:  # Overwrites any existing file.
        pickle.dump(obj, output, pickle.HIGHEST_PROTOCOL)
    
def load_objects(file):
    with open(file, 'rb') as input:
        return pickle.load(input)
    
def vocabDict(vocab, n): # handles empty word sets
    nk = 0
    # n = the number of word postions for this document type
    probability = ((nk + 1) / (n + len(vocab)))

    newDict = {"count": 0, "probability":probability}
    vdict = dict((el,newDict) for el in vocab)
    return vdict

def wordCount(doc, vocab, n): # get actual word count for each word and create dict frame to track the info
    counts = dict(Counter(doc.tolist()[0].split()))
    for key, value in counts.items():
        nk = value
        probability = ((nk + 1) / (n + len(vocab)))
        newDict = {"count": nk, "probability": probability}
        counts[key] = newDict
    emptyVocab = vocabDict(vocab, n)
    dicts = [merge_two_dicts(emptyVocab, counts)]
    return dicts
    

def saveFrame(df, name):
    df.to_csv(name+".csv", index=False, sep=",", header=True)
    save_it_all(df, name+".pkl")

In [101]:
# # load in files 
# # read file line by line 
# fileName = "../forumTraining.data"
# data = read(fileName)
# len(data)
# sep_data, typesInfo, docsInfo = seperator(data)
# # write(sep_data) not needed the raw csv it makes IS BROKEN
# # print(len(list(set(typesInfo))))
# # print(list(set(typesInfo)))
# dataFrameRaw = pd.DataFrame({"Type": typesInfo, "Document": docsInfo})
# saveFrame(dataFrameRaw, "raw_training_data.csv")

In [102]:
raw_training_data = pd.read_csv("./raw_training_data.csv", sep=",")
raw_training_data.head()

,Type,Document
0,atheism,alt atheism faq atheist resources archive name...
1,atheism,alt atheism faq introduction to atheism archiv...
2,atheism,re gospel dating in article mimsy umd edu mang...
3,atheism,re university violating separation of church s...
4,atheism,re soc motss et al princeton axes matching fun...


### Trianing data raw is held in the dataframe "raw_training_data"

In [103]:
# create the raw vocabulary set:
raw_vocabulary = " ".join(raw_training_data["Document"].tolist()) # concatinates all documents in the data frame
raw_vocabulary = list(set(raw_vocabulary.split()))
print(len(raw_vocabulary))
save_it_all(raw_vocabulary, "./raw_vocabulary.pkl")

73712


### The raw vocabulary is saved in the above cell and refrenced as "raw_vocabulary" 

# This is the learning step: make it dynamic!
For each class cj (document type) in C
    
    1. Docsj ← training documents for which the classification is cj
    
    2. Probability estimate of a particular class: P(cj) = |Docsj| / |training documents|
    
    3. Textj ← create a single document per class (concatenate all Docsj)
    
    4. n = total number of word positions in Textj
    
    5. For each word wk in Vocabulary nk = number of times wk occurs in Textj
    
    6. Estimate of word occurrence for particular document type: P(wk | cj) = (nk + 1) / (n + |Vocabulary|)
    
Probability of kth word in vocabulary, given a document of type j

In [104]:
# load in docs you want to use:
## trainign data set for use:
trainFrame = pd.read_csv("./raw_training_data.csv", sep=",")

## vocabulary training set for use:
vocabulary = load_objects("./raw_vocabulary.pkl")

# validate:
print(type(vocabulary))
trainFrame.head()

<class 'list'>


,Type,Document
0,atheism,alt atheism faq atheist resources archive name...
1,atheism,alt atheism faq introduction to atheism archiv...
2,atheism,re gospel dating in article mimsy umd edu mang...
3,atheism,re university violating separation of church s...
4,atheism,re soc motss et al princeton axes matching fun...


In [105]:
# single docs per class
types = list(set(trainFrame["Type"]))
singleFrame = pd.DataFrame({"Type":types , "Document": None})
# singleFrame.head(20)
for docType in singleFrame["Type"]:
    #print(trainFrame["Document"][trainFrame["Type"]==docType])
    singleFrame["Document"][singleFrame["Type"]== docType] = " ".join(trainFrame["Document"][trainFrame["Type"]==docType].tolist())
singleFrame.head(30)

,Type,Document
0,mideastpolitics,re islam borders was israel misisipi to ganges...
1,politics,re media horrified at perot investigating bush...
2,religion,re religion and homosexuality magarret magnus ...
3,mac,re nutek faces apple s wrath article read in a...
4,forsale,motorcycle wanted followup to kedz wpi wpi edu...
5,hockey,superstars and attendance was teemu selanne wa...
6,mswindows,re is kermit available for windows from articl...
7,xwindows,th international obfuscated c code contest ope...
8,medicine,info needed gaucher s disease i have a yr old ...
9,electronics,re help with tracking device in article fba e ...


In [106]:
# make a copy of the single frame 
rawDataInfoFrame = singleFrame.copy()

In [107]:
# probability estimate of the class
rawDataInfoFrame["ClassProbability"] = None
for docType in rawDataInfoFrame["Type"]:
    rawDataInfoFrame["ClassProbability"][rawDataInfoFrame["Type"]== docType] = len(trainFrame["Document"][trainFrame["Type"]== docType].tolist()) / trainFrame["Type"].count()


In [108]:
rawDataInfoFrame.head()

,Type,Document,ClassProbability
0,mideastpolitics,re islam borders was israel misisipi to ganges...,0.0499424
1,politics,re media horrified at perot investigating bush...,0.0411759
2,religion,re religion and homosexuality magarret magnus ...,0.0333835
3,mac,re nutek faces apple s wrath article read in a...,0.0511821
4,forsale,motorcycle wanted followup to kedz wpi wpi edu...,0.051802


In [109]:
# get the word postions:
rawDataInfoFrame["WordPositions"] = None
for docType in rawDataInfoFrame["Type"]:
    rawDataInfoFrame["WordPositions"][rawDataInfoFrame["Type"]== docType] = len(rawDataInfoFrame["Document"][rawDataInfoFrame["Type"]== docType].tolist()[0].split())
    


In [110]:
rawDataInfoFrame.head(20)

,Type,Document,ClassProbability,WordPositions
0,mideastpolitics,re islam borders was israel misisipi to ganges...,0.0499424,272488
1,politics,re media horrified at perot investigating bush...,0.0411759,202625
2,religion,re religion and homosexuality magarret magnus ...,0.0333835,129611
3,mac,re nutek faces apple s wrath article read in a...,0.0511821,97182
4,forsale,motorcycle wanted followup to kedz wpi wpi edu...,0.051802,69761
5,hockey,superstars and attendance was teemu selanne wa...,0.0531303,153949
6,mswindows,re is kermit available for windows from articl...,0.0506508,95238
7,xwindows,th international obfuscated c code contest ope...,0.0525104,175024
8,medicine,info needed gaucher s disease i have a yr old ...,0.052599,170336
9,electronics,re help with tracking device in article fba e ...,0.0523333,115503


In [111]:
# word count 
rawDataInfoFrame["WordCount"] = None
for docType in rawDataInfoFrame["Type"]:
    n = rawDataInfoFrame["WordPositions"][rawDataInfoFrame["Type"] == docType].tolist()[0]
    print(n)
    rawDataInfoFrame["WordCount"][rawDataInfoFrame["Type"] == docType] = wordCount(rawDataInfoFrame["Document"][rawDataInfoFrame["Type"]==docType], vocabulary, n)
    

In [114]:
rawDataInfoFrame.head()
rawDataInfoFrame["WordCount"][rawDataInfoFrame["Type"] == "graphics"].tolist()[0]

{'hepis': {'count': 0, 'probability': 5.072537283149031e-06},
 'calculates': {'count': 0, 'probability': 5.072537283149031e-06},
 'rectifier': {'count': 0, 'probability': 5.072537283149031e-06},
 'philemon': {'count': 0, 'probability': 5.072537283149031e-06},
 'wretched': {'count': 0, 'probability': 5.072537283149031e-06},
 'davpa': {'count': 0, 'probability': 5.072537283149031e-06},
 'bsplinesurface': {'count': 1, 'probability': 1.0145074566298062e-05},
 'tarurs': {'count': 0, 'probability': 5.072537283149031e-06},
 'sealift': {'count': 0, 'probability': 5.072537283149031e-06},
 'rapings': {'count': 0, 'probability': 5.072537283149031e-06},
 'truman': {'count': 0, 'probability': 5.072537283149031e-06},
 'balthazar': {'count': 0, 'probability': 5.072537283149031e-06},
 'intruder': {'count': 0, 'probability': 5.072537283149031e-06},
 'cobus': {'count': 0, 'probability': 5.072537283149031e-06},
 'xgetrgbcolormap': {'count': 0, 'probability': 5.072537283149031e-06},
 'cosequences': {'coun

In [115]:
saveFrame(rawDataInfoFrame, "./raw_data_info_frame")

In [64]:
rawDataInfoFrameEdit = rawDataInfoFrame.drop(columns=["WordPositions"])

In [26]:
saveFrame(rawDataInfoFrameEdit, "./raw_data_info_frame_V2")
rawDataInfoFrameEdit.head()

,Type,Document,ClassProbability,WordCount
0,mideastpolitics,re islam borders was israel misisipi to ganges...,-2.99688,"{'hepis': {'count': 0, 'probability': -12.7547..."
1,politics,re media horrified at perot investigating bush...,-3.1899,"{'hepis': {'count': 0, 'probability': -12.5293..."
2,religion,re religion and homosexuality magarret magnus ...,-3.39969,"{'hepis': {'count': 0, 'probability': -12.2225..."
3,mac,re nutek faces apple s wrath article read in a...,-2.97236,"{'hepis': {'count': 0, 'probability': -12.0487..."
4,forsale,motorcycle wanted followup to kedz wpi wpi edu...,-2.96033,"{'hepis': {'count': 0, 'probability': -11.8739..."
